In [1]:
%env DATA_BUCKET=tbd-2023z-304098-data
%env GEN_OUTPUT_DIR=/tmp/tpc-di
%env REPO_ROOT=/home/jupyter/git/tbd-tpc-di/

env: DATA_BUCKET=tbd-2023z-304098-data
env: GEN_OUTPUT_DIR=/tmp/tpc-di
env: REPO_ROOT=/home/jupyter/git/tbd-tpc-di/


In [ ]:
!pip3.8 install typer[All]==0.9.0 google-cloud-storage==2.13.0

## Install SDKMAN for setting up JVM 8 enviroment

In [ ]:
%%bash
curl -s https://get.sdkman.io | bash

## Install and set as default JVM 8

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 8.0.392-amzn
sdk use java 8.0.392-amzn

## Check if JVM 8 is used

In [5]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
java -version

openjdk version "1.8.0_392"
OpenJDK Runtime Environment Corretto-8.392.08.1 (build 1.8.0_392-b08)
OpenJDK 64-Bit Server VM Corretto-8.392.08.1 (build 25.392-b08, mixed mode)


## Clone tbd-tpc-di repo

In [6]:
%%bash
mkdir -p git && cd git
git clone https://github.com/thai-chicken/tbd-tpc-di.git
cd tbd-tpc-di
git pull
git checkout notebook

fatal: destination path 'tbd-tpc-di' already exists and is not an empty directory.


Already up to date.


Already on 'notebook'


D	tests/fact_watches_dates_proper_relation.sql
Your branch is up to date with 'origin/notebook'.


## Generate input dataset (run this cell below from the terminal!!!)
It should take approx. 15min with scale factor set to 100 and generate approx. 10GiB

In [ ]:
%%script bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd /home/jupyter/git/tbd-tpc-di/tools/ 
java -jar DIGen.jar -sf 10 -o /tmp/tpc-di

## Install and setup JVM 11

In [8]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 11.0.21-amzn
sdk use java 11.0.21-amzn -y


java 11.0.21-amzn is already installed.

Using java version 11.0.21-amzn in this shell.


## Load staging

In [9]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd $REPO_ROOT
python3.8 tpcdi.py --output-directory $GEN_OUTPUT_DIR --stage $DATA_BUCKET

:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8f25649b-c864-4cb5-8879-1409c34c6e00;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.17.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
:: resolution report :: resolve 694ms :: artifacts dl 22ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.17.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 from central in [default]
	----------------------

24/01/04 22:12:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/04 22:12:15 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/01/04 22:12:16 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/01/04 22:12:24 WARN Client: Same path resource file:///root/.ivy2/jars/com.databricks_spark-xml_2.12-0.17.0.jar added multiple times to distributed cache.
24/01/04 22:12:24 WARN Client: Same path resource file:///root/.ivy2/jars/commons-io_commons-io-2.11.0.jar added multiple times to distributed cache.
24/01/04 22:12:24 WARN Client: Same path resource file:///root/.ivy2/jars/org.glassfish.jaxb_txw2-3.0.2.jar added multiple times to distributed cache.
24/01/04 22:12:24 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.ws.xmlschema_xmlschema-core-2.3.0.jar added multiple times to distributed cache.
24/01/04 22:12:24 WARN Client: Same path resource file:///root/.ivy2/jars/org.scala-lang.modules_scala

## Run dbt ELT

In [10]:
%%bash
cd $REPO_ROOT
dbt deps
dbt run

22:18:19  Running with dbt=1.7.3
22:18:19  Installing dbt-labs/dbt_utils
22:18:20  Installed from version 1.1.1
22:18:20  Up to date!
22:18:24  Running with dbt=1.7.3
22:18:25  Registered adapter: spark=1.7.1
22:18:25  Found 44 models, 4 tests, 17 sources, 0 exposures, 0 metrics, 553 macros, 0 groups, 0 semantic models
22:18:25  
:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-22f21f88-292d-4b32-aed2-323dee8b367e;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.17.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
:: resolution report :: resolve 571ms :: artifacts dl 20ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.17.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 from central in [default]
	----------------------

24/01/04 22:18:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/04 22:18:34 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/01/04 22:18:34 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/01/04 22:18:42 WARN Client: Same path resource file:///root/.ivy2/jars/com.databricks_spark-xml_2.12-0.17.0.jar added multiple times to distributed cache.
24/01/04 22:18:42 WARN Client: Same path resource file:///root/.ivy2/jars/commons-io_commons-io-2.11.0.jar added multiple times to distributed cache.
24/01/04 22:18:42 WARN Client: Same path resource file:///root/.ivy2/jars/org.glassfish.jaxb_txw2-3.0.2.jar added multiple times to distributed cache.
24/01/04 22:18:42 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.ws.xmlschema_xmlschema-core-2.3.0.jar added multiple times to distributed cache.
24/01/04 22:18:42 WARN Client: Same path resource file:///root/.ivy2/jars/org.scala-lang.modules_scala

## Run dbt tests

In [11]:
%%bash
cd $REPO_ROOT
dbt deps
dbt test

22:27:47  Running with dbt=1.7.3
22:27:48  Installing dbt-labs/dbt_utils
22:27:48  Installed from version 1.1.1
22:27:48  Up to date!
22:27:52  Running with dbt=1.7.3
22:27:53  Registered adapter: spark=1.7.1
22:27:53  Found 44 models, 4 tests, 17 sources, 0 exposures, 0 metrics, 553 macros, 0 groups, 0 semantic models
22:27:53  
:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3fcec813-3c46-422b-815b-97d9de5f26b1;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.17.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
:: resolution report :: resolve 555ms :: artifacts dl 21ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.17.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 from central in [default]
	----------------------

24/01/04 22:27:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/04 22:28:04 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/01/04 22:28:04 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/01/04 22:28:14 WARN Client: Same path resource file:///root/.ivy2/jars/com.databricks_spark-xml_2.12-0.17.0.jar added multiple times to distributed cache.
24/01/04 22:28:14 WARN Client: Same path resource file:///root/.ivy2/jars/commons-io_commons-io-2.11.0.jar added multiple times to distributed cache.
24/01/04 22:28:14 WARN Client: Same path resource file:///root/.ivy2/jars/org.glassfish.jaxb_txw2-3.0.2.jar added multiple times to distributed cache.
24/01/04 22:28:14 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.ws.xmlschema_xmlschema-core-2.3.0.jar added multiple times to distributed cache.
24/01/04 22:28:14 WARN Client: Same path resource file:///root/.ivy2/jars/org.scala-lang.modules_scala

# Test Spark

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TBD-TPC-DI-setup") \
    .master("yarn") \
    .enableHiveSupport() \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ccb06429-1b11-4007-8aea-a563e4eb9ebb;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.17.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
:: resolution report :: resolve 607ms :: artifacts dl 34ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.17.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 from central in [default]
	----------------------

24/01/05 00:15:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/05 00:15:42 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/01/05 00:15:42 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/01/05 00:15:51 WARN Client: Same path resource file:///root/.ivy2/jars/com.databricks_spark-xml_2.12-0.17.0.jar added multiple times to distributed cache.
24/01/05 00:15:51 WARN Client: Same path resource file:///root/.ivy2/jars/commons-io_commons-io-2.11.0.jar added multiple times to distributed cache.
24/01/05 00:15:51 WARN Client: Same path resource file:///root/.ivy2/jars/org.glassfish.jaxb_txw2-3.0.2.jar added multiple times to distributed cache.
24/01/05 00:15:51 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.ws.xmlschema_xmlschema-core-2.3.0.jar added multiple times to distributed cache.
24/01/05 00:15:51 WARN Client: Same path resource file:///root/.ivy2/jars/org.scala-lang.modules_scala

In [ ]:
spark.sql("show databases").show()

24/01/05 00:16:20 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
+-----------+
|  namespace|
+-----------+
|     bronze|
|    default|
|demo_bronze|
|  demo_gold|
|demo_silver|
|      digen|
|       gold|
|     silver|
+-----------+



In [ ]:
spark.sql("use demo_gold")

DataFrame[]

In [ ]:
spark.sql("show tables").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|demo_gold|         dim_account|      false|
|demo_gold|          dim_broker|      false|
|demo_gold|         dim_company|      false|
|demo_gold|        dim_customer|      false|
|demo_gold|            dim_date|      false|
|demo_gold|        dim_security|      false|
|demo_gold|           dim_trade|      false|
|demo_gold|  fact_cash_balances|      false|
|demo_gold|fact_cash_transac...|      false|
|demo_gold|       fact_holdings|      false|
|demo_gold|          fact_trade|      false|
|demo_gold|        fact_watches|      false|
+---------+--------------------+-----------+



# Calculate layers' tables

In [42]:
print(f"{'Layer':>15} | {'# Tables':>10}")
print("-"*16 + "+" + "-"*11)
for db_row in spark.sql("show databases").collect():
    db_name = db_row.namespace
    spark.sql(f"use {db_name}")
    n_tables = spark.sql("show tables").count()
    print(f"{db_name:>15} | {n_tables:>10}")
        

          Layer |   # Tables
----------------+-----------
         bronze |          0
        default |          0
    demo_bronze |         17
      demo_gold |         12
    demo_silver |         14
          digen |         17
           gold |          0
         silver |          0


# Testing

In [ ]:
df_pandas = spark.sql("SELECT * FROM demo_gold.fact_watches LIMIT 10").toPandas()

In [ ]:
df_pandas.head(2)

In [ ]:
spark.sql('''SELECT sk_trade_id, COUNT(*) cnt
            FROM demo_gold.fact_trade
            GROUP BY sk_trade_id
            HAVING cnt > 1
    ''').show()